<a href="https://colab.research.google.com/github/yashphulfagar/IITMBS/blob/main/TDS/Visualizing_network_data_with_Kumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U scikit-learn scipy matplotlib
!pip install scikit-network
import pandas as pd
import numpy as np
import sknetwork.clustering
import sknetwork.utils
from scipy.sparse import csr_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.6 MB/s eta 0:00:00


In [ ]:
# !rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

total 1155968
drwxr-xr-x 1 root root      4096 Jul 30 11:19 .
drwxr-xr-x 1 root root      4096 Jul 30 11:13 ..
drwxr-xr-x 4 root root      4096 Jul 27 13:36 .config
-rw-r--r-- 1 root root 251516986 Jul 29 13:25 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Jul 27 13:37 sample_data
-rw-r--r-- 1 root root 310734144 Jul 29 13:25 title.akas.tsv.gz
-rw-r--r-- 1 root root 175310103 Jul 29 13:25 title.basics.tsv.gz
-rw-r--r-- 1 root root 446110311 Jul 29 13:25 title.principals.tsv.gz


In [ ]:
title=pd.read_csv('title.basics.tsv.gz',sep='\t',low_memory=False).set_index('tconst')[['titleType','primaryTitle','startYear']]

In [ ]:
title.shape

(10041816, 3)

In [ ]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
title["startYear"].dtype

dtype('O')

In [ ]:
pd.to_numeric(title["startYear"], downcast= 'float' )

ValueError: ignored

In [ ]:
title["startYear"] > '2000'

tconst
tt0000001    False
tt0000002    False
tt0000003    False
tt0000004    False
tt0000005    False
             ...  
tt9916848     True
tt9916850     True
tt9916852     True
tt9916856     True
tt9916880     True
Name: startYear, Length: 10041816, dtype: bool

In [ ]:
cast=pd.read_csv('title.principals.tsv.gz',sep='\t', usecols=['tconst','nconst','category'])
cast=cast[cast.category.isin({'actor','actress'})]
cast.head()

,tconst,nconst,category
11,tt0000005,nm0443482,actor
12,tt0000005,nm0653042,actor
16,tt0000007,nm0179163,actor
17,tt0000007,nm0183947,actor
21,tt0000008,nm0653028,actor


In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
# This is what the network looks like
cast.head()

,tconst,nconst,category
24,tt0000009,nm0063086,actress
25,tt0000009,nm0183823,actor
26,tt0000009,nm1309758,actor
850,tt0000502,nm0215752,actor
851,tt0000502,nm0252720,actor


In [ ]:
# Explore the regions we have data for (e.g. IN, US, etc)
region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=False).set_index('titleId')['region']
region.value_counts().head(10)